In [ ]:
using Test
using LinearAlgebra
using Plots

In [ ]:
# Defining function to minimize
function f(x)
    sum = 0;
    for i = 1:length(x)
        sum += x[i]
    end

    return sum;
end

In [ ]:
@test f([1,2,3,4,5,6]) == 21

In [ ]:
# Defining gradient of f.
function grad_f(x)
    grad = ones(length(x))
    return grad  
end

In [ ]:
@test grad_f([1,1,1,1,1]) == [1, 1, 1, 1, 1]

In [ ]:
# Making the infinity norm of a matrix.
function infty_norm(x)
    max = 0;
    for i = 1:length(x)
        if( max < abs(x[i]) )
            max = abs(x[i]);
        end
    end

    return max;
end

In [ ]:
# infty_norm([1000, 1, 2, 3, 4, 5, -1002]) == 1000
@test infty_norm([1000, 1, 2, 3, 4, 5, -1002]) == 1002

In [ ]:
# Choosing the descent directions.
function descent_generate(grad)
    d = randn(Float64, length(grad));
    while( transpose(grad) ⋅ d  >= 0 )
        d = randn(Float64, length(grad));
    end

    return d;
end

In [ ]:
grad = transpose(grad_f([1,2,3,4,5]))
d = descent_generate(grad)
@test grad ⋅ d < 0

In [ ]:
# Making the function of linear search for finding the step
function stepper(x, d)
    λ = 1;
    while( f(x+λ*d) >= f(x) )
        λ /= 2;
    end 

    return λ
end

In [ ]:
# Descent Algoritm
function descent_algo(x)
    ϵ           = 1e-6;
    x_old       = copy(x);
    max_iter    = 1000;
    for i = 1:max_iter
        d       = descent_generate(grad_f(x));
        λ       = stepper(x, d);
        x       = x + λ*d;
        if( infty_norm(x-x_old) < ϵ )
            break;
        end
        x_old = copy(x);
    end

    return x;
end

In [ ]:
initial_x   = [1, 1, 1];
result      = descent_algo(initial_x);

println("Result: ", result);

In [ ]:
# Defining Rosenbrok function
function ROSENBROK(n::Int64, x::Vector{Float64})
    if n % 2 == 1
        return -1;
    end
    value = 0;
    for i = 1:Int(n/2)
        value += (x[2i] - x[2i-1]^2 )^2 + ( x[2i-1] - 1 )^2
    end

    return value;
end    

In [ ]:
function GRAD_ROSENBROK_DERIVADA(x::Vector{Float64})
    n       = length(x);
    grad    = zeros(n);
    for i = 1:Int(n/2)
        grad[2*i-1] = -4*(x[2*i] - x[2*i-1]*x[2*i-1])*x[2*i-1] + 2*(x[2*i-1]-1);
        grad[2*i]   = 2*(x[2*i] - x[2*i-1]*x[2*i-1]);
    end

    return grad
end

In [ ]:
# Making the function with Gradient Descent and Armijo's condition.
# Implement the "extended" linear search.
function ARMIJO(x::Vector{Float64}, M::Int64, a::Float64)
    ϵ = 1e-6;
    k = 0;
    d = GRAD_ROSENBROK_DERIVADA(x); # Gradient Descent condition.
    while ( (infty_norm(d) > ϵ) && (k < M) )
        d       = - GRAD_ROSENBROK_DERIVADA(x);
        t       = 1;
        f_xtd   = ROSENBROK(2, x+t*d);
        f_x     = ROSENBROK(2, x);
        armijo  = -a * dot(d, d);
        while( f_xtd > f_x + armijo*t && t > ϵ )
            t   /= 2;
        end
        x       = x + t*d;
        k       = k + 1;
    end

    return x;
end

In [ ]:
x = [2.0, 2.0];
M = 100;
a = 0.9;
ok = ARMIJO(x, M, a)

min = [1.0, 1.0]
display(ROSENBROK(2, ok))
display(ROSENBROK(2, min))

In [ ]:
num_of_samples = Int(1000);
xlim = zeros(Int, num_of_samples);
ylim = zeros(Float64, num_of_samples);
#display(x);
#display(y);

x       = [2.0, 2.0];
x_aux   = [0.0, 0.0];
a       = 0.9;

for i = 1:num_of_samples
    xlim[i]     = i;
    x_aux       = ARMIJO(x, i, a);
    value       = ROSENBROK(2, x_aux);
    ylim[i]     = value
end

ylim
display(plot(xlim, ylim));


In [ ]:
function gradient_descent(α::Float64, σ::Float64, ϵ::Float64,
                          M::Int64, x_k::Vector{Float64}, ∇::Function, f::Function)
    k   = 0;
    g_k = ∇(x_k);
    μ   = infty_norm(g_k);

    while( (μ >= ϵ) && (k <= M) )
        λ_k = 1;
        # Armijo's condition:
        ∧   = α * dot(g_k, g_k);
        while( f(x_k + λ_k * g_k) > f(x_k) + λ_k*∧ )
            λ_k = σ * λ_k;
        end

        x_k = x_k + λ_k * g_k; 
        g_k = ∇(x_k);
        μ   = infty_norm(g_k);
        k   = k + 1;
    end

    return x;
end

In [ ]:
function quad(x::Vector{Float64})
    sum = 0;
    
end